# How to pick the best strike and expiration for trading options

## Parse historic options data

## Load data into Optopsy

## Analyze strategy performance

In [10]:
# import and setup
import os
from pathlib import Path
import glob
import pandas as pd
import optopsy as op

In [26]:
files = glob.glob(str(Path(Path(os.getcwd()).parents[0], "data", os.path.join("rut-eod", "*.csv"))))
dfs = []
for fl in files:
    df = pd.read_csv(fl)
    dfs.append(df)
pd.concat(dfs).to_csv(Path(Path(os.getcwd()).parents[0], "data","rut_historic.csv"), index=False)

In [28]:
# Loading data into Optopsy
rut_chains = op.csv_data(
    Path(Path(os.getcwd()).parents[0], "data","rut_historic.csv"),
    underlying_symbol=1,
    underlying_price=4,
    option_type=8,
    expiration=6,
    quote_date=0,
    strike=7,
    bid=14,
    ask=15
)

/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/optopsy/datafeeds.py:42: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data["expiration"] = pd.to_datetime(data.expiration, infer_datetime_format=True)
/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/optopsy/datafeeds.py:43: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data["quote_date"] = pd.to_datetime(data.quote_date, infer_datetime_format=True)


In [ ]:
# Analyze strategies with Optopsy
op.short_calls(rut_chains).round(2)

### Let’s break down the key output:

- dte_range: The range of days remaining until the options expire. For example, (0, 7] includes all options that will expire in 0 to 7 days based on the quote date.
- otm_pct_range: Range of how far out of the money the options are, expressed as a percentage. For example, (-0.5, -0.45] means the options are between 50% and 45% out of the money.
- count: The number of options contracts that fall within the specified dte_range and otm_pct_range. For instance, there are 666 options contracts that expire in 0 to 7 days and are 50% to 45% out of the money.
- mean: This is the average percent change of the options within the specified dte_range and otm_pct_range.

In [ ]:
op.short_strangles(
    rut_chains, 
    dte_interval=60, 
    max_entry_dte=70, 
    exit_dte=10,
    otm_pct_interval=0.01,
    max_otm_pct=0.10
).round(2)

There are 3,070 short strangles found between 0 and 60 days to expiration. The first leg, usually the put, groups contracts between 9% and 10% out of the money and the first leg between 3% and 4% out of the money. The mean return achieved by exiting this position at 10 days to expiration is 39%.